In [1]:
import pandas as pd
import pandahouse as ph

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. 
Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 заданий.

## Задание 1
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов

#### Подключимся к Clickhouse и выполним запрос

In [5]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [8]:
q = '''

    SELECT
        DISTINCT(st_id),
        sum(correct) as correct_sum

    FROM 
        default.peas

    GROUP BY 
        st_id

    HAVING 
        correct_sum >= 20

    ORDER BY 
        correct_sum DESC
    
    '''

In [9]:
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test.shape[0]

136

Получается количество очень усердных учеников составляет - 136 человек
____

## Задание 2
Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:   
ARPU   
ARPAU   
CR в покупку   
СR активного пользователя в покупку   
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике


In [10]:
q_2 = '''
    
    -- ARPU = Доход/Число пользователей
    -- ARPAU = Доход активных пользователей/Число активных пользователей
    -- CR в покупку = Число пользователей, которые купили курс/Число пользователей
    -- СR активного пользователя в покупку = Число активных пользователей, которые купили курс / Число активных пользователей
    -- CR пользователя из активности по математике в покупку курса по математике = Число активных пользователей по математике, которые купили курс/Число активных пользователей по математике


SELECT 
    test_grp,
    round(SUM(df.money)/uniqExact(ds.st_id), 2) as ARPU,
    round(SUMIf(df.money, active.is_Active > 10)/uniqIf(ds.st_id, active.is_Active > 10), 2) as ARPAU,
    round(100*uniqIf(ds.st_id, df.money > 0)/uniqExact(ds.st_id), 2) as CR,
    round(100*uniqIf(ds.st_id, active.is_Active > 10 and df.money > 0)/uniqIf(ds.st_id, active.is_Active > 10) ,2) as CR_active,
    round(100*uniqIf(ds.st_id, active.is_Active_Math > 1 and df.money > 0)/
                                                            uniqIf(ds.st_id, active.is_Active_Math > 1), 2) as CR_active_math
FROM default.studs as ds
LEFT JOIN (
            SELECT 
                st_id, SUM(correct) as is_Active, SUMIf(correct, subject = 'Math') as is_Active_Math
            FROM default.peas
            GROUP BY st_id
            ) as active  
ON ds.st_id = active.st_id
LEFT JOIN default.final_project_check as df
ON ds.st_id = df.st_id
GROUP BY test_grp
    
    '''

In [11]:
q_test_2 = ph.read_clickhouse(query=q_2, connection=connection_default)
q_test_2

,test_grp,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.98,10393.70,4.92,11.02,10.20
1,pilot,11508.47,29739.58,10.85,26.04,14.29
